# Hadoop Streaming assignment 3: Name Count
Make WordCount program for all the names in the dataset. Name is a word with the following properties:

* The first character is not a digit (other characters can be digits).
* The first character is uppercase, all the other characters that are letters are lowercase.
* There are less than 0.5% occurrences of this word, when this word regardless to its case appears in the dataset and the condition (2) is not met.

Order by quantity, most popular first, output format:

<code>name <tab> count</code>

The result is the 5th line in the output.

The result on the sample dataset:

<code>french 5742</code>

If you want to deploy the environment on your own machine, please use bigdatateam/yarn-notebook Docker container.

In [162]:
%%writefile test.dat

1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde! For The Horde!
1	For Horde! For Horde! For Horde! For Horde! For Horde! For Horde! For Horde!
2	for the horde HoRde hordE
3	1one 2two

Overwriting test.dat


In [164]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        cond1 = word[0].isalpha()
        cond2 = not word[0].islower() and word[1:].islower()      
        if (cond1):
            print "%s\t%d\t%d" % (word.lower(), cond2, 1)

Overwriting mapper1.py


In [165]:
cat test.dat | python2 ./mapper1.py | sort | head

for	0	1
for	1	1
for	1	1
for	1	1
for	1	1
for	1	1
for	1	1
for	1	1
for	1	1
for	1	1


In [172]:
%%writefile reducer1.py

import sys

current_key=None
total_sum = 0
cond2_sum = 0
cond3 = lambda total_sum, cond2_sum: float(total_sum-cond2_sum)/total_sum < 0.005

for line in sys.stdin:
    try:
        word, cond2, count = line.strip().split('\t', 2)
        cond2 = int(cond2)
        count = int(count)
    except ValueError as e:
        continue
    
    if current_key != word:        
        if current_key and cond3(total_sum, cond2_sum):
            print "%s\t%d" % (current_key, cond2_sum)   
        current_key = word
        total_sum = 0
        cond2_sum = 0         
    total_sum += count
    cond2_sum += cond2*count

if current_key and cond3(total_sum, cond2_sum):
    print "%s\t%d" % (current_key, cond2_sum)

Overwriting reducer1.py


In [173]:
cat test.dat | python2 ./mapper1.py | sort | python2 ./reducer1.py

for	252
the	245


In [184]:
%%writefile mapper2.py

import sys

for line in sys.stdin:
    try:
        key, value = line.strip().split('\t', 1)
        value = int(value)
        print "%d\t%s" % (value, key)
    except ValueError as e:
        continue        

Overwriting mapper2.py


In [185]:
cat test.dat | python2 ./mapper1.py | sort | python2 ./reducer1.py | python2 ./mapper2.py

252	for
245	the


In [186]:
%%writefile reducer2.py

import sys

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)
        count = int(count)
        print "%s\t%d" % (word, count)
    except ValueError as e:
        continue

Overwriting reducer2.py


In [188]:
cat test.dat | python2 ./mapper1.py | sort | python2 ./reducer1.py | python2 ./mapper2.py | sort -nr | python2 ./reducer2.py

for	252
the	245


In [193]:
%%bash

TMP_DIR="tmp_"
OUT_DIR="out_"
LOGS="logs.txt"

hdfs dfs -rm -r -skipTrash ${OUT_DIR}* > /dev/null
hdfs dfs -rm -r -skipTrash ${TMP_DIR}* > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="NameCount Step1" \
    -D mapreduce.job.reduces=8 \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${TMP_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="NameCount Step2" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-nr" \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${TMP_DIR} \
    -output ${OUT_DIR} > /dev/null
    
hdfs dfs -cat "out_/*" | head -5 | tail -1

french	5742


rm: `out_*': No such file or directory
19/04/17 18:04:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/04/17 18:04:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/04/17 18:04:15 INFO mapred.FileInputFormat: Total input files to process : 1
19/04/17 18:04:15 INFO mapreduce.JobSubmitter: number of splits:2
19/04/17 18:04:15 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1555514672898_0004
19/04/17 18:04:15 INFO impl.YarnClientImpl: Submitted application application_1555514672898_0004
19/04/17 18:04:15 INFO mapreduce.Job: The url to track the job: http://91bfcd6a3d82:8088/proxy/application_1555514672898_0004/
19/04/17 18:04:15 INFO mapreduce.Job: Running job: job_1555514672898_0004
19/04/17 18:04:22 INFO mapreduce.Job: Job job_1555514672898_0004 running in uber mode : false
19/04/17 18:04:22 INFO mapreduce.Job:  map 0% reduce 0%
19/04/17 18:04:38 INFO mapreduce.Job:  map 41% reduce 0%
19/04/17 18:04:44 INFO mapreduce.Job:  ma